** EXTRACCION DE DATOS **

In [8]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime

In [9]:
# Versiones
print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

numpy==2.1.3
pandas==2.2.3
requests==2.32.3


In [121]:
# Constantes comunes
URL = "https://apidatos.ree.es" # URL base de la API
HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es",
}

# Diccionario de geo_ids y nombres de regiones
region_ids = {
    "8741": "peninsular",
    "8742": "canarias",
    "8743": "baleares",
    "8744": "ceuta",
    "8745": "melilla",
    "4": "andalucia",
    "5": "aragon",
    "6": "cantabria",
    "7": "castilla la mancha",
    "8": "castilla y leon",
    "9": "cataluña",
    "10": "pais vasco",
    "11": "principado de asturias",
    "13": "comunidad de madrid",
    "14": "comunidad de navarra",
    "15": "comunidad de valenciana",
    "16": "extremadura",
    "17": "galicia",
    "20": "la rioja",
    "21": "region de murcia"
}

In [11]:
filtro_balance = '/es/datos/balance/balance-electrico'

filtro_demanda = '/es/datos/demanda/evolucion'
filtro_general = '/es/datos/demanda/ire-general'
filtro_industria = '/es/datos/demanda/ire-industria'
filtro_servicios = '/es/datos/demanda/ire-servicios'

filtro_generacion = "/es/datos/generacion/estructura-generacion"
filtro_renovable = "/es/datos/generacion/evolucion-renovable-no-renovable"

In [12]:
def Balance_electrico(filtro_balance, URL, HEADERS):

    filtro_balance

    endpoint = f"{URL}/{filtro_balance}"

    # Inicializamos un DataFrame vacío para almacenar los datos
    df_balance = pd.DataFrame()

    # Iteramos sobe los años de 2019 a 2024 y extraemos los datos de balance
    # de cada región para cada año
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            data = response.json()
            included = data.get('included', [])

            registros = []
            # Extraemos los datos de balance
            # para cada grupo de energía
            # y cada indicador
            for grupo in included:
                group_name = grupo['attributes']['title']
                contenidos = grupo['attributes'].get('content', [])

                for item in contenidos:
                    indicador = item['attributes']['title']
                    valores = item['attributes'].get('values', [])
                    # Extraemos los valores para cada indicador
                    # y los almacenamos en el DataFrame
                    for punto in valores:
                        registros.append({
                            'fecha': punto['datetime'],
                            'valor': punto['value'],
                            'tipo': indicador,
                            'energia': group_name,
                            'region': region_name,
                        })
            # Convertimos los registros a un DataFrame
            # y lo concatenamos al DataFrame principal
            df_balance_year = pd.DataFrame(registros)
            df_balance = pd.concat([df_balance, df_balance_year], ignore_index=True)

    return df_balance

In [13]:
def demanda_evolucion(filtro_demanda, URL, HEADERS):

    filtro_demanda

    endpoint = f"{URL}/{filtro_demanda}"
    # Inicializamos un df vacío para almacenar los datos
    df_demanda = pd.DataFrame()
    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                # Extraemos los datos de balance
                # para cada grupo de energía
                # y cada indicador
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'indicador': indicador,
                        'region': region_name,
                    })

            df_demanda_year = pd.DataFrame(registros)
            df_demanda = pd.concat([df_demanda, df_demanda_year], ignore_index=True)

    return df_demanda



In [14]:
def demanda_ire_general(filtro_general, URL, HEADERS):
    
    filtro_general

    endpoint = f"{URL}/{filtro_general}"

    # Inicializamos un DataFrame vacío
    df_ire_general = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                        
                    })

            df_ire_year = pd.DataFrame(registros)
            df_ire_general = pd.concat([df_ire_general, df_ire_year], ignore_index=True)

    return df_ire_general

In [15]:
def demanda_ire_industria(filtro_industria, URL, HEADERS):

    filtro_industria

    endpoint = f"{URL}/{filtro_industria}"

    # Inicializamos un DataFrame vacío
    df_ire_industria = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                    })

            df_ire_industria_year = pd.DataFrame(registros)
            df_ire_industria = pd.concat([df_ire_industria, df_ire_industria_year], ignore_index=True)

    return df_ire_industria

In [16]:
def demanda_ire_servicios(filtro_servicios, URL, HEADERS):

    filtro_servicios

    endpoint = f"{URL}/{filtro_servicios}"

    # Inicializamos un DataFrame vacío
    df_ire_servicios = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                    })

            df_ire_servicios_year = pd.DataFrame(registros)
            df_ire_servicios = pd.concat([df_ire_servicios, df_ire_servicios_year], ignore_index=True)

    return df_ire_servicios

In [17]:
def generacion(filtro_generacion, URL, HEADERS):

    filtro_generacion

    endpoint = f"{URL}{filtro_generacion}"

    df_generacion = pd.DataFrame()
    ## Iterar sobre los años
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        # Iterar sobre los geo_ids y nombres de regiones
        # para cada año y geo_id
        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                try:
                    data = response.json()
                    included = data.get('included', [])

                    if included:
                        registros = []

                        # Iterar sobre los grupos de datos en 'included'
                        for grupo in included:
                            group_name = grupo['attributes'].get('title', 'Desconocido')
                            group_type = grupo['attributes'].get('type', 'Desconocido')
                            contenidos = grupo['attributes'].get('values', [])

                            # Iterar sobre los valores de generación
                            for punto in contenidos:
                                registros.append({
                                    'fecha': punto.get('datetime', 'Desconocido'),
                                    'valor': punto.get('value', 0),
                                    'porcentaje': punto.get('percentage', 0),
                                    'indicador': group_name,
                                    'region': region_name,
                                    'tipo': group_type,
                                })

                        df_generacion_year = pd.DataFrame(registros)
                        df_generacion = pd.concat([df_generacion, df_generacion_year], ignore_index=True)

                except Exception as e:
                    print("Contenido de la respuesta:", response.text)

            else:
                print(f"Error en la solicitud para {region_name} {year}. Código: {response.status_code}")

    return df_generacion

In [18]:
def fronteras(URL, HEADERS):

    # Lista de países
    lista_paises = ['francia-frontera', 'portugal-frontera', 'marruecos-frontera', 'andorra-frontera']

    datos_intercambios = []

    for pais in lista_paises:
        # Endpoint para intercambios
        filtro_intercambio = f"/es/datos/intercambios/{pais}"
        endpoint = f"{URL}{filtro_intercambio}"
        # Iterar sobre los años y extraer datos de cada año
        # y cada país para el año 2015 al 2025
        for year in range(2015, 2026):

            if year < 2025:
                start_date = f"{year}-01-01T00:00"
                end_date = f"{year}-12-31T23:59"
                TIME_TRUNC = "day"
            else:
                start_date = f"{year}-01-01T00:00"
                end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
                TIME_TRUNC = "day"

            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                data = response.json()
                # Verificar si hay datos disponibles
                valores = data["included"][0]["attributes"].get("values", [])
                if valores:
                    df = pd.DataFrame(valores)
                    df["pais"] = pais
                    datos_intercambios.append(df)
            else:
                print(f"Error en la solicitud para {pais} en {year}. Código: {response.status_code}")

    # Unir todos los datos en un único DataFrame
    df_fronteras = pd.concat(datos_intercambios, ignore_index=True)

    return df_fronteras

In [19]:
def extraccion():

    # Función para extraer datos de la API de REE empleando las funciones para cada tipo de dato.

    print('Extrayendo datos del Balance Electrico...')
    df_balance = Balance_electrico(filtro_balance, URL, HEADERS)
    
    print('')

    print('Extrayendo datos de la Demanda Electrica...')
    df_demanda = demanda_evolucion(filtro_demanda, URL, HEADERS)

    print('')

    print('Extrayendo datos de la IRE General...')
    df_ire_general = demanda_ire_general(filtro_general, URL, HEADERS)
    
    print('')

    print('Extrayendo datos de la IRE Industria...')
    df_ire_industria = demanda_ire_industria(filtro_industria, URL, HEADERS)

    print('')

    print('Extrayendo datos de la IRE Servicios...')
    df_ire_servicios = demanda_ire_servicios(filtro_servicios, URL, HEADERS)

    print('')

    print('Extrayendo datos de la Generación Eléctrica...')
    df_generacion = generacion(filtro_generacion, URL, HEADERS)

    print('')

    print('Extrayendo datos de los intercambios entre países...')
    df_fronteras = fronteras(URL, HEADERS)

    return {
        "balance": df_balance,
        "demanda": df_demanda,
        "ire_general": df_ire_general,
        "ire_industria": df_ire_industria,
        "ire_servicios": df_ire_servicios,
        "generacion": df_generacion,
        "fronteras": df_fronteras,
    }

In [120]:
dataframes = extraccion() # Tarda aprox 17 minutos en hacer toda la extraccion de todos los datos

Extrayendo datos del Balance Electrico...

Extrayendo datos de la Demanda Electrica...

Extrayendo datos de la IRE General...

Extrayendo datos de la IRE Industria...

Extrayendo datos de la IRE Servicios...

Extrayendo datos de la Generación Eléctrica...

Extrayendo datos de los intercambios entre países...


** LIMPIEZA DE DATOS **

In [213]:
df_balance = dataframes['balance'].copy()
print('Balance:')
display(df_balance)

df_demanda = dataframes['demanda'].copy()
print('Demanda:')
display(df_demanda)

df_ire_general = dataframes['ire_general'].copy()
print('IRE General:')
display(df_ire_general)

df_ire_industria = dataframes['ire_industria'].copy()
print('IRE Industria:')
display(df_ire_industria)

df_ire_servicios = dataframes['ire_servicios'].copy()
print('IRE Servicios:')
display(df_ire_servicios)

df_generacion = dataframes['generacion'].copy()
print('Generacion:')
display(df_generacion)

df_fronteras = dataframes['fronteras'].copy()
print('Fronteras:')
display(df_fronteras)

Balance:


,fecha,valor,tipo,energia,region
0,2015-01-01T00:00:00.000+01:00,71096.101,Hidráulica,Renovable,peninsular
1,2015-01-02T00:00:00.000+01:00,101890.114,Hidráulica,Renovable,peninsular
2,2015-01-03T00:00:00.000+01:00,82919.726,Hidráulica,Renovable,peninsular
3,2015-01-04T00:00:00.000+01:00,69056.368,Hidráulica,Renovable,peninsular
4,2015-01-05T00:00:00.000+01:00,104975.898,Hidráulica,Renovable,peninsular
...,...,...,...,...,...
1695715,2025-05-01T00:00:00.000+02:00,525874.544,Demanda en b.c.,Demanda,region de murcia
1695716,2025-05-02T00:00:00.000+02:00,573531.974,Demanda en b.c.,Demanda,region de murcia
1695717,2025-05-03T00:00:00.000+02:00,552763.137,Demanda en b.c.,Demanda,region de murcia
1695718,2025-05-04T00:00:00.000+02:00,526498.877,Demanda en b.c.,Demanda,region de murcia


Demanda:


,fecha,valor,indicador,region
0,2015-01-01T00:00:00.000+01:00,603961.474,Demanda,peninsular
1,2015-01-02T00:00:00.000+01:00,715038.679,Demanda,peninsular
2,2015-01-03T00:00:00.000+01:00,698648.512,Demanda,peninsular
3,2015-01-04T00:00:00.000+01:00,663533.254,Demanda,peninsular
4,2015-01-05T00:00:00.000+01:00,714784.323,Demanda,peninsular
...,...,...,...,...
75555,2025-05-01T00:00:00.000+02:00,525874.544,Demanda,region de murcia
75556,2025-05-02T00:00:00.000+02:00,573531.974,Demanda,region de murcia
75557,2025-05-03T00:00:00.000+02:00,552763.137,Demanda,region de murcia
75558,2025-05-04T00:00:00.000+02:00,526498.877,Demanda,region de murcia


IRE General:


...

,fecha,valor,porcentaje,indicador,region
0,2015-01-01T00:00:00.000+01:00,115.701000,0.526620,Índice servicios,peninsular
1,2015-02-01T00:00:00.000+01:00,108.047000,0.504358,Índice servicios,peninsular
2,2015-03-01T00:00:00.000+01:00,115.520000,0.522961,Índice servicios,peninsular
3,2015-04-01T00:00:00.000+02:00,109.634000,0.507696,Índice servicios,peninsular
4,2015-05-01T00:00:00.000+02:00,121.254000,0.540774,Índice servicios,peninsular
...,...,...,...,...,...
9835,2025-02-01T00:00:00.000+01:00,-2.215705,1.000000,Variación mensual,region de murcia
9836,2025-03-01T00:00:00.000+01:00,2.939563,1.000000,Variación mensual,region de murcia
9837,2025-01-01T00:00:00.000+01:00,1.185036,1.000000,Variación mensual corregida,region de murcia
9838,2025-02-01T00:00:00.000+01:00,1.385936,1.000000,Variación mensual corregida,region de murcia


Generacion:


,fecha,valor,porcentaje,indicador,region,tipo
0,2015-01-01T00:00:00.000+01:00,71096.101,0.114430,Hidráulica,peninsular,Renovable
1,2015-01-02T00:00:00.000+01:00,101890.114,0.147343,Hidráulica,peninsular,Renovable
2,2015-01-03T00:00:00.000+01:00,82919.726,0.117828,Hidráulica,peninsular,Renovable
3,2015-01-04T00:00:00.000+01:00,69056.368,0.101494,Hidráulica,peninsular,Renovable
4,2015-01-05T00:00:00.000+01:00,104975.898,0.145461,Hidráulica,peninsular,Renovable
...,...,...,...,...,...,...
1209895,2025-05-01T00:00:00.000+02:00,571425.544,1.000000,Generación total,region de murcia,Generación total
1209896,2025-05-02T00:00:00.000+02:00,630842.874,1.000000,Generación total,region de murcia,Generación total
1209897,2025-05-03T00:00:00.000+02:00,567613.537,1.000000,Generación total,region de murcia,Generación total
1209898,2025-05-04T00:00:00.000+02:00,559770.477,1.000000,Generación total,region de murcia,Generación total


Fronteras:


,value,percentage,datetime,pais
0,-3384.261,0.122156,2015-01-01T00:00:00.000+01:00,francia-frontera
1,-60.199,0.002045,2015-01-02T00:00:00.000+01:00,francia-frontera
2,-19.245,0.000649,2015-01-03T00:00:00.000+01:00,francia-frontera
3,-31.131,0.000995,2015-01-04T00:00:00.000+01:00,francia-frontera
4,-2.958,0.000098,2015-01-05T00:00:00.000+01:00,francia-frontera
...,...,...,...,...
15087,-350.200,1.000000,2025-04-26T00:00:00.000+02:00,andorra-frontera
15088,-303.760,1.000000,2025-04-27T00:00:00.000+02:00,andorra-frontera
15089,-87.490,1.000000,2025-04-28T00:00:00.000+02:00,andorra-frontera
15090,-0.060,0.857143,2025-04-29T00:00:00.000+02:00,andorra-frontera


In [180]:
def limpieza_balance(df):
    df_balance = df.copy()
    # Cambiamos el formato de fecha y en valor, trabajamos con MWh ya que se ven mejor
    df_balance['fecha'] = pd.to_datetime(df_balance['fecha'].astype(str).str.split('T').str[0])
    df_balance['valor'] = df_balance['valor']/1000

    # La columna de las regiones parece que está repitiendo los datos, con lo que eliminamos los duplicados que haya en 
    # el resto de columnas sin tener en cuenta esa

    df_balance_sin_duplicados = df_balance.drop_duplicates(subset=['fecha', 'valor', 'tipo', 'energia'])

    return df_balance_sin_duplicados

In [166]:
def limpieza_demanda(df1_demanda, df2_demanda, df3_demanda, df4_demanda):

    # Le damos un formato correcto y limpiamos las fechas
    df1_demanda['fecha'] = pd.to_datetime(df1_demanda['fecha'].astype(str).str.split('T').str[0])
    df2_demanda['fecha'] = pd.to_datetime(df2_demanda['fecha'].astype(str).str.split('T').str[0])
    df3_demanda['fecha'] = pd.to_datetime(df3_demanda['fecha'].astype(str).str.split('T').str[0])
    df4_demanda['fecha'] = pd.to_datetime(df4_demanda['fecha'].astype(str).str.split('T').str[0])

    df1_demanda['valor'] = df1_demanda['valor']/1e3

    # Limpiamos los duplicados en cada df obtenido

    df1_demanda_sin_duplicados = df1_demanda.drop_duplicates(subset=['fecha', 'valor', 'indicador'])
    df2_demanda_sin_duplicados = df2_demanda.drop_duplicates(subset=['fecha', 'valor', 'porcentaje', 'indicador'])
    df3_demanda_sin_duplicados = df3_demanda.drop_duplicates(subset=['fecha', 'valor', 'porcentaje', 'indicador'])
    df4_demanda_sin_duplicados = df4_demanda.drop_duplicates(subset=['fecha', 'valor', 'porcentaje', 'indicador'])
    
    # Se juntan los df que contienen datos IRE (IRE GENERAL, IRE INDUSTRIA E IRE SERVICIOS)
    df_ire = pd.concat([df2_demanda_sin_duplicados, df3_demanda_sin_duplicados, df4_demanda_sin_duplicados], ignore_index=True)

    df_ire = df_ire[~df_ire['indicador'].isin(['Variación mensual corregida', 'Variación mensual'])]
    # Ordenamos por fecha
    df1_demanda.sort_values('fecha', inplace=True)
    df_ire.sort_values('fecha', inplace=True)

    return df1_demanda, df_ire

In [181]:
def limpieza_generacion(df):

    df_generacion = df.copy()
    # Cambiamos el formato de fecha (nos quedamos con YYYY-MM-DD)
    df_generacion['fecha'] = pd.to_datetime(df_generacion['fecha'].astype(str).str.split('T').str[0])

    # Convertimos Wh a MWh
    df_generacion['valor'] = df_generacion['valor'] / 1e3

    # Quitamos las filas con tipo 'Generación total'
    df_generacion = df_generacion[df_generacion['tipo'] != 'Generación total']

    # Eliminamos duplicados ignorando la columna 'region'
    df_generacion_sin_duplicados = df_generacion.drop_duplicates(
        subset=['fecha', 'valor', 'porcentaje', 'indicador', 'tipo']
    )

    return df_generacion_sin_duplicados

In [ ]:
def limpieza_fronteras(df):
    # Cambiamos el formato de fecha y en valor, trabajamos con kWh ya que se ven mejor

    df_fronteras = df.copy()

    df_fronteras['datetime'] = pd.to_datetime(df_fronteras['datetime'].str.split('T').str[0])
    df_fronteras['value'] = df_fronteras['value']/1e3

    # Cambiamos los nombres de las columnas que se entiende mejor

    df_fronteras.rename(columns={
        'datetime': 'fecha',
        'value': 'valor',
        'percentage': 'porcentaje'}, inplace=True)

    #Quitamos duplicados:
    df_fronteras_limpio = df_fronteras.drop_duplicates(
        subset=['fecha', 'pais', 'valor', 'porcentaje']
    )

    return df_fronteras_limpio

In [214]:
df_balance_limpio = limpieza_balance(df_balance)
df_demanda_limpia, df_ire_limpia = limpieza_demanda(df_demanda, df_ire_general, df_ire_industria, df_ire_servicios)
df_generacion_limpia = limpieza_generacion(df_generacion)
df_fronteras_limpias = limpieza_fronteras(df_fronteras)

In [215]:
print('Limpieza de datos:')
print('Balance:')
display(df_balance_limpio)
print('Demanda:')
display(df_demanda_limpia)
print('IRE:')
display(df_ire_limpia)
print('Generación:')
display(df_generacion_limpia)
print('Fronteras:')
display(df_fronteras_limpias)

Limpieza de datos:
Balance:


,fecha,valor,tipo,energia,region
0,2015-01-01,71.096101,Hidráulica,Renovable,peninsular
1,2015-01-02,101.890114,Hidráulica,Renovable,peninsular
2,2015-01-03,82.919726,Hidráulica,Renovable,peninsular
3,2015-01-04,69.056368,Hidráulica,Renovable,peninsular
4,2015-01-05,104.975898,Hidráulica,Renovable,peninsular
...,...,...,...,...,...
1639152,2025-05-01,525.874544,Demanda en b.c.,Demanda,peninsular
1639153,2025-05-02,573.531974,Demanda en b.c.,Demanda,peninsular
1639154,2025-05-03,552.763137,Demanda en b.c.,Demanda,peninsular
1639155,2025-05-04,526.498877,Demanda en b.c.,Demanda,peninsular


Demanda:


,fecha,valor,indicador,region
0,2015-01-01,603.961474,Demanda,peninsular
6205,2015-01-01,603.961474,Demanda,galicia
5840,2015-01-01,603.961474,Demanda,extremadura
5475,2015-01-01,603.961474,Demanda,comunidad de valenciana
5110,2015-01-01,603.961474,Demanda,comunidad de navarra
...,...,...,...,...
75059,2025-05-05,425.357560,Demanda,comunidad de valenciana
75184,2025-05-05,425.357560,Demanda,extremadura
73184,2025-05-05,425.357560,Demanda,peninsular
74559,2025-05-05,425.357560,Demanda,pais vasco


IRE:


,fecha,valor,porcentaje,indicador,region
0,2015-01-01,118.061,0.538442,Índice general,peninsular
504,2015-01-01,99.816,0.451245,Índice industria corregido,peninsular
996,2015-01-01,104.004,0.468711,Índice servicios corregido,peninsular
984,2015-01-01,115.701,0.526620,Índice servicios,peninsular
12,2015-01-01,101.203,0.457435,Índice general corregido,peninsular
...,...,...,...,...,...
485,2025-03-01,97.987,0.449580,Índice general corregido,peninsular
977,2025-03-01,92.917,0.437771,Índice industria corregido,peninsular
974,2025-03-01,112.514,0.547697,Índice industria,peninsular
1466,2025-03-01,112.585,0.521705,Índice servicios,peninsular


Generación:


,fecha,valor,porcentaje,indicador,region,tipo
0,2015-01-01,71.096101,0.114430,Hidráulica,peninsular,Renovable
1,2015-01-02,101.890114,0.147343,Hidráulica,peninsular,Renovable
2,2015-01-03,82.919726,0.117828,Hidráulica,peninsular,Renovable
3,2015-01-04,69.056368,0.101494,Hidráulica,peninsular,Renovable
4,2015-01-05,104.975898,0.145461,Hidráulica,peninsular,Renovable
...,...,...,...,...,...,...
1171846,2025-05-01,1.607600,0.002813,Residuos renovables,peninsular,Renovable
1171847,2025-05-02,1.688800,0.002677,Residuos renovables,peninsular,Renovable
1171848,2025-05-03,1.713500,0.003019,Residuos renovables,peninsular,Renovable
1171849,2025-05-04,1.672850,0.002988,Residuos renovables,peninsular,Renovable


Fronteras:


,valor,porcentaje,fecha,pais
0,-3.384261,0.122156,2015-01-01,francia-frontera
1,-0.060199,0.002045,2015-01-02,francia-frontera
2,-0.019245,0.000649,2015-01-03,francia-frontera
3,-0.031131,0.000995,2015-01-04,francia-frontera
4,-0.002958,0.000098,2015-01-05,francia-frontera
...,...,...,...,...
15087,-0.350200,1.000000,2025-04-26,andorra-frontera
15088,-0.303760,1.000000,2025-04-27,andorra-frontera
15089,-0.087490,1.000000,2025-04-28,andorra-frontera
15090,-0.000060,0.857143,2025-04-29,andorra-frontera


CARGAR A LA BASE DE DATOS

In [185]:
from dotenv import load_dotenv
import mysql.connector
import os

load_dotenv()

db_host = os.getenv("DATABASE_HOST")
db_user = os.getenv("DATABASE_USER")
db_psw = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")

def db_connect():
    # Conexión MySQL
    conn = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_psw,
        database=db_name
    )

    return conn

In [216]:
import pandas as pd

def carga_balance(df_balance_limpio):
    print("Cargando datos...")
    # Creamos la conexion a la base de datos a traves del script
    # db_connect() que establece la conexion con dicha BD
    conn = db_connect()
    cursor = conn.cursor()
    # Ordenamos el df para que las columnas tengan el mismo orden
    # que en la base de datos
    nuevo_balance = ['fecha', 'tipo', 'energia', 'region', 'valor']
    df_balance_limpio = df_balance_limpio[nuevo_balance]

    # Insertar los datos en la tabla balance
    for i, row in df_balance_limpio.iterrows():
        cursor.execute("""
            INSERT INTO balance (fecha, tipo, energia, region, valor)
            VALUES (%s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE valor = VALUES(valor)
        """, (row['fecha'], row['tipo'], row['energia'], row['region'], row['valor']))
    
    # Confirmamos la carga
    conn.commit()

    # Cerramos el cursor y la conexión
    cursor.close()
    conn.close()

    return True

In [217]:
def carga_ire(df_ire_limpio):
    print("Cargando datos...")
    # Creamos la conexion a la base de datos a traves del script
    # db_connect() que establece la conexion con dicha BD
    conn = db_connect()
    cursor = conn.cursor()
    # Ordenamos el df para que las columnas tengan el mismo orden
    # que en la base de datos
    nuevo_ire = ['fecha', 'indicador', 'region', 'valor', 'porcentaje']
    df_ire_limpio = df_ire_limpio[nuevo_ire]

    # Aqui cargamos los datos fila por fila a diferencia del archivo anterior
    # debido a que hay una cantidad menor de datos que subir a la base de datos
    for _, row in df_ire_limpio.iterrows():
        cursor.execute("""
            INSERT IGNORE INTO demanda_ire_general (fecha, indicador, region, valor, porcentaje)
            VALUES (%s, %s, %s, %s, %s)
        """, (row['fecha'], row['indicador'], row['region'], row['valor'], row['porcentaje']))
    # Confirmamos la sentencia
    conn.commit()
    # Cerramos el cursor y la conexion con la base de datos
    cursor.close()
    conn.close()

    return True

In [218]:
def carga_demanda(df_demanda_limpio):
    print("Cargando datos...")
    # Creamos la conexion a la base de datos a traves del script
    # db_connect() que establece la conexion con dicha BD
    conn = db_connect()
    cursor = conn.cursor()
    # Ordenamos el df para que las columnas tengan el mismo orden
    # que en la base de datos
    nuevo_demanda = ['fecha', 'indicador', 'region', 'valor']
    df_demanda_limpio = df_demanda_limpio[nuevo_demanda]
    
    # Dividir el dataframe en lotes para que se suba a la base de datos
    # con mayor facilidad
    batch_size = 1000
    num_batches = len(df_demanda_limpio) // batch_size + 1

    for i in range(num_batches):
        batch = df_demanda_limpio.iloc[i * batch_size: (i + 1) * batch_size]
        data_batch = [tuple(row) for row in batch.to_numpy()]
        # Ejecutamos la sentencia SQL con executemany para cargar los lotes de datos
        cursor.executemany("""
            INSERT IGNORE INTO demanda_evolucion (fecha, indicador, region, valor)
            VALUES (%s, %s, %s, %s)
        """, data_batch)
    # Confirmar la sentencia
    conn.commit()
    # Cerramos el curson y la conexion a la BD
    cursor.close()
    conn.close()
    
    return True

In [219]:
def carga_fronteras(df_fronteras_limpio):
    print("Cargando datos...")
    # Creamos la conexion a la base de datos a traves del script
    # db_connect() que establece la conexion con dicha BD
    conn = db_connect()
    cursor = conn.cursor()
    # Ordenamos el df para que las columnas tengan el mismo orden
    # que en la base de datos
    nuevo_fronteras = ['fecha', 'pais', 'valor', 'porcentaje']
    df_fronteras_limpio = df_fronteras_limpio[nuevo_fronteras]
    df_fronteras_limpio.head()

    # Aqui cargamos los datos fila por fila debido a que hay 
    # una cantidad menor de datos que subir a la base de datos
    for _, row in df_fronteras_limpio.iterrows():
        # Ejecutamos la sentencia
        cursor.execute("""
            INSERT IGNORE INTO fronteras (fecha, pais, valor, porcentaje)
            VALUES (%s, %s, %s, %s)
        """, (row['fecha'], row['pais'], row['valor'], row['porcentaje']))
    # Confirmamos la carga
    conn.commit()
    # Cerramos el cursor y la conexion con la BD
    cursor.close()
    conn.close()

    return True

In [220]:
def carga_generacion(df_generacion_limpio):
    print("Cargando datos...")
    # Creamos la conexion a la base de datos a traves del script
    # db_connect() que establece la conexion con dicha BD
    conn = db_connect()
    cursor = conn.cursor()
    # Ordenamos el df para que las columnas tengan el mismo orden
    # que en la base de datos
    nuevo_estructura = ['fecha', 'indicador', 'region', 'tipo', 'valor', 'porcentaje']
    df_generacion_limpio = df_generacion_limpio[nuevo_estructura]
    # Aqui cargamos los datos fila por fila debido a que hay 
    # una cantidad menor de datos que subir a la base de datos
    for _, row in df_generacion_limpio.iterrows():
        # Ejecutamos la sentencia
        cursor.execute("""
            INSERT IGNORE INTO estructura_generacion (fecha, indicador, region, tipo, valor, porcentaje)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['fecha'], row['indicador'], row['region'], row['tipo'], row['valor'], row['porcentaje']))
    # Confirmamos la carga
    conn.commit()
    # Cerramos el cursor y la conexion con la BD
    cursor.close()
    conn.close()

    return True

In [225]:
carga = carga_balance(df_balance_limpio)
if carga == True:
    print("Carga de datos completada.")
else:
    print("Error en la carga de datos.")
    print("Carga de datos fallida.")

Cargando datos...
Carga de datos completada.


In [226]:
carga = carga_ire(df_ire_limpia)
if carga == True:
    print("Carga de datos completada.")
else:
    print("Error en la carga de datos.")
    print("Carga de datos fallida.")

Cargando datos...
Carga de datos completada.


In [227]:
carga = carga_demanda(df_demanda_limpia)
if carga == True:
    print("Carga de datos completada.")
else:
    print("Error en la carga de datos.")
    print("Carga de datos fallida.")

Cargando datos...
Carga de datos completada.


In [228]:
carga = carga_fronteras(df_fronteras_limpias)
if carga == True:
    print("Carga de datos completada.")
else:
    print("Error en la carga de datos.")
    print("Carga de datos fallida.")

Cargando datos...
Carga de datos completada.


In [229]:
carga = carga_generacion(df_generacion_limpia)
if carga == True:
    print("Carga de datos completada.")
else:
    print("Error en la carga de datos.")
    print("Carga de datos fallida.")

Cargando datos...
Carga de datos completada.
